### Práctica de Formatos de datos: CSV y JSON

#### Ejercicio 1[5 puntos]
Considerar el archivo __Contaminación.csv__. El archivo contiene la información recogida por las estaciones de control de calidad del aire durante el año 2019.

Una descripción detallada de la estructura de los datos se puede encontrar en el siguiente enlace: https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/Aire/Ficheros/Interprete_ficheros_%20calidad_%20del_%20aire_global.pdf

La información contenida por columnas es la siguiente:
* __PROVINCIA__: Código numérico que representa la provincia. 
* __MUNICIPIO__: Código numérico que representa el municipio. 
* __ESTACIÓN__: Código numérico de hasta dos dígitos que representa la estación. Consultar ANEXO I de la documentación
* __MAGNITUD__: Código numérico de hasta dos dígitos que representa la magnitud.Consultar ANEXO II de la documentación
* __PUNTO MUESTREO__: Código numérico formado por 3 subcódigos. El primer código representa la estación, el segundo código representa la magnitud, y el tercer código representa la técnica de medida.
* __MES__: Mes de la toma de datos. Valor entre 1 y 12.
* __D01-D31__: Medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente.
* __V01-V31__: Validez de las medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente. Puede valer V(Válido), N(No válido), no se conoce(vacío) o bien otra cadena(se considera incorrecto).


Se pide crear un programa que realice las siguientes operaciones:
* Leer los datos del archivo __Contaminación.csv__
* Transformar los datos leidos de manera que aquellos valores que no sean válidos ("N") su valor sea sustituido por 0.
* Obtener y  mostrar por pantalla para cada tipo de contaminante (columna Magnitud) y para cada estación (columna ESTACIÓN), la suma total de la contaminación que se ha producido durante el año 2019, es decir:

|Estación|Contaminante1| Contaminante2| Contaminante3| ....Contaminanten|
|----|-------------|-------------|-------------|-------------|
|Estación1|Subtotal11 |--- |--- |Subtotal1n|  
|Estación2|--- |--- |--- |--- | 
|   ...|--- |--- |--- |--- |     
|Estaciónn|Subtotal1n |--- |--- |Subtotalnn| 
     
* Generar un documento __Salida.csv__ donde cada línea del documento contiene la siguiente información:
    Estación1, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
    Estaciónm, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.
    

Para leer los archivos usa un código como el siguiente:

In [ ]:
codEst={"35":"Pza. del Carmen","4":"Pza. de España","39":"Barrio del Pilar","8":"Escuelas Aguirre",
        "38":"Cuatro Caminos","11":"Av. Ramón y Cajal","40":"Vallecas","16":"Arturo Soria",
        "17":"Villaverde Alto","18":"C/ Farolillo","36":"Moratalaz","24":"Casa de Campo","27":"Barajas",
        "47":"Méndez Álvaro","48":"Pº. Castellana","49":"Retiro","50":"Pza. Castilla","54":"Ensanche Vallecas",
        "55":"Urb. Embajada (Barajas)","56":"Plaza Elíptica","57":"Sanchinarro","58":"El Pardo",
        "59":"Parque Juan Carlos I","60":"Tres Olivos"}

In [ ]:
codCont={"1":"Dióxido de Azufre","6":"Monóxido de Carbono","7":"Monóxido de Nitrógeno","8":"Dióxido de Nitrógeno",
         "9":"Partículas < 2.5 μm","10":"Partículas < 10 μm","12":"Óxidos de Nitrógeno","14":"Ozono",
         "20":"Tolueno","30":"Benceno","35":"Etilbenceno","37":"Metaxileno","38":"Paraxileno","39":"Ortoxileno",
         "42":"Hidrocarburos totales(hexano)","43":"Metano","44":"Hidrocarburos no metánicos (hexano)"}

In [ ]:
import csv
def leerEstacionesContaminacion():
    csvarchivo = open('Contaminación.csv',encoding="utf8",errors='ignore')
    entrada = csv.reader(csvarchivo, delimiter=";")
    
    #Diccionario {estacion:{magnitud:cantidad}}
    estaciones = {}

    #[2] -> estación
    #[3] -> magnitud
    #[7] en adelante-> días y validez

    for lista in entrada:
        if(entrada.line_num != 1):
            if codEst[lista[2]] not in estaciones.keys():
                magnitudes={}
            if codCont[lista[3]] not in magnitudes.keys():
                magnitudes[codCont[lista[3]]] = 0
            for i in range(7,len(lista),2):
                if(lista[i+1] == 'V'):#Si el dato es válido
                    magnitudes[codCont[lista[3]]] += float(lista[i])
        else:
            continue
        estaciones[codEst[lista[2]]] = magnitudes   
    csvarchivo.close()
    return estaciones

In [ ]:
#Muestra para cada estación la suma de los contaminantes
#estaciones->Diccionario {estacion:{magnitud:cantidad}}
def mostrarDatosContaminacion(estaciones):
    cabecera="****************"
    
    listaContaminantes=list(codCont.values())
    print(cabecera)
    
    for estacion, magnitud in estaciones.items():
        print("\n")
        print("ESTACIÓN: "+estacion+"\n")
        i = 0
        for tipoCont, suma in magnitud.items():
            contaminante=""
            if (tipoCont == listaContaminantes[i]):
                contaminante=listaContaminantes[i]+": "+str(suma)
            else:
                while (tipoCont != listaContaminantes[i]):
                    contaminante=listaContaminantes[i]+": 0"
                    print(contaminante) 
                    i+=1
                contaminante=listaContaminantes[i]+": "+str(suma)  
                    
            i+=1
            print(contaminante)
        
        while (i<len(listaContaminantes)):
            contaminante=listaContaminantes[i]+": 0"
            print(contaminante)
            i+=1
        print("\n")
        print(cabecera)

In [ ]:
#Escribe los datos de cada estación(contaminante y subtotal)
#estaciones->Diccionario {estacion:{magnitud:cantidad}}
def escribirDatosContaminacion(estaciones):
    csvFileObj = open("contaminacionEstaciones.csv", "w",encoding="utf8")
    csvWriter = csv.writer(csvFileObj, lineterminator='\n')

    #Estación1,Contaminante1,Subtotal1,Contaminante2,Subtotal2,........,Contaminanten,Subtotaln
    for estacion, magnitud in estaciones.items():
        res=[]
        res.append(str(estacion))
        for tipo, con in magnitud.items():
            res.append(str(tipo))
            res.append(con)
        csvWriter.writerow(res)

    csvFileObj.close()

In [ ]:
estacionesCont = leerEstacionesContaminacion()
mostrarDatosContaminacion(estacionesCont)
escribirDatosContaminacion(estacionesCont)

#### Ejercicio 2[5 puntos]
Considerar los archivos  __Estaciones.csv__ y __Museos.json__ que incluye información sobre las estaciones de control de la calidad del aire y sobre los museos de Madrid respectivamente.

El objetivo de este ejercicio es utilizar la información obtenida en el ejercicio 1 de forma que para cada uno de los museos que aparece en el archivo dado realice las siguientes operaciones:
* Obtener las 3 estaciones más cercanas a cada museo.
* Obtener y mostrar por pantalla para cada museo la siguiente información donde ValorX es el subtotal del nivel de contaminación más alto de todos los contaminantes medidos en la estaciónX, y TipoContaminanteX es el tipo del contaminante cuyo valor es mostrado en la celda anterior.

|Museo|Estación1|TipoContaminante1| Estación2|TipoContaminante2| Estación3|TipoContaminante3|
|----|-------------|-------------|-------------|----|----|----|
|Museo1|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |
|Museo2|--- |--- |--- |--- | --- | --- | 
|   ...|--- |--- |--- |--- | --- | --- |     
|Museon|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |

* Generar un documento __Salida.json__ con la siguiente estructura:

Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.

Para leer los archivos usa un código como el siguiente:

In [ ]:
#import json
#leer = json.loads(open('Museos.json',encoding="utf8").read())
#print(leer)

En los documentos __Estación.csv__ y __Museos.json__ aparece la información de geolocalización en forma de longitud y latitud tanto de las estaciones como de los museos. En este sentido, para calcular la distancia entre dos puntos dadas sus coordenadas se utilizará la siguiente función en Python:

In [ ]:
#Calcula distancia
import math

def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [ ]:
import csv
def leerEstacionesLocalizacion():
    csvarchivo = open('Estaciones.csv',encoding="utf8",errors='ignore')
    entrada = csv.reader(csvarchivo, delimiter=";")
    
    #Diccionario {estacion:[latitud, longitud]}
    estaciones = {}

    #[1] -> estacion
    #[24] -> longitud
    #[25]-> latitud

    for lista in entrada:
        if(entrada.line_num != 1):
            if codEst[lista[1]] not in estaciones.keys():
                posicion=[float(lista[25]),float(lista[24])]
        else:
            continue
        estaciones[codEst[lista[1]]] = posicion   
    csvarchivo.close()
    return estaciones

In [ ]:
import json

def leerMuseos():
    leer = json.loads(open('Museos.json',encoding="utf8").read())
    
    #diccionario={museo:[latitud,longitud]}
    museos={}
    for i,museo in leer.items():
        if(i == "@graph"):
            i=0
            while(i<len(museo)):
                title=""
                for t,l in museo[i].items():
                    location=[]
                    if(t == "title"):
                        title=l
                    if(t == "location"):
                        for m,lo in l.items():
                            location.append(lo)
                            museos[title]=location                    
                i+=1
    return museos

In [ ]:
import operator
#estaciones={estacion:[latitud,longitud]}, museos={museo:[latitud,longitud]}
def estacionesCercanas(estaciones, museos):
    #Buscar las 3 estaciones más cercanas para cada museo
    buscado={}
    #Recorre las estaciones para cada museo
    for museo,locM in museos.items():
        distancias={}
        for e,locE in estaciones.items():
            d=haversine(locM[0],locM[1],locE[0],locE[1])
            distancias[e]=d
        #Ordena las estaciones por distancia de menor a mayor
        distancias = sorted(distancias.items(), key=operator.itemgetter(1))
        #Elimina todas las estaciones menos las 3 más cercanas
        del(distancias[3:len(distancias)])
        buscado[museo]=distancias
 
    return buscado

In [ ]:
#Buscar mayor contaminante para la estación dada.

#contaminantesEstacion={estacion:{magnitud,cantidad}
#estacion= nombre de la estacion para la que se quiere buscar el mayor contaminante
def mayorContaminante(estacion,contaminantesEstacion):
    
    mayorCont=[]
    
    magnitud = contaminantesEstacion[estacion]
    #Ordena las magnitudes de la estacion de manor a mayor
    magnitud = sorted(magnitud.items(), key=operator.itemgetter(1))
    #Deja el elemento mayor
    del(magnitud[0:len(magnitud)-1])
    #Guarda el tipo de magnitud(contaminante)
    mayorCont.append(magnitud[0][0])
    #Guarda la cantidad
    mayorCont.append(magnitud[0][1])
    
    return mayorCont

In [ ]:
#Muestra por pantalla para cada museo las 3 estaciones más cercanas y su mayor contaminante
def mostrarMuseosContaminacion(cercanas, contaminantesEstacion):
    
    separador="****************"
    print(separador)
    for m,e in cercanas.items():
        print("\n")
        text="MUSEO: "+m+"\n"
        print(text)
        #Estación 1
        text = "Estación 1: "+e[0][0]
        print(text)
        mayor=mayorContaminante(e[0][0],contaminantesEstacion)
        text =mayor[0]+": "+str(mayor[1])
        print(text)
        #Estación 2
        text = "Estación 2: "+e[1][0]
        print(text)
        mayor=mayorContaminante(e[1][0],contaminantesEstacion)
        text =mayor[0]+": "+str(mayor[1])
        print(text)
        #Estación 3
        text = "Estación 3: "+e[2][0]
        print(text)
        mayor=mayorContaminante(e[2][0],contaminantesEstacion)
        text =mayor[0]+": "+str(mayor[1])
        print(text)
        
        print("\n")
        print(separador)

In [ ]:

def escribirMuseos(estacionesCercanas, contaminantesEstacion):
    
    museosJson={"año":2019,"fuente":"Ayuntamiento de Madrid"}
    museos=[]
    for m,est in estacionesCercanas.items():
        museo={}
        
        museo["museo"]=m
        for e in est:
            estaciones={}
            estaciones["Nombre"]=e[0]
            mayor=mayorContaminante(e[0],contaminantesEstacion)
            estaciones["Valor"]=mayor[1]
            estaciones["Tipo"]=mayor[0]
            
            museo[e[0]]= estaciones
            
        museos.append(museo)
        
    museosJson["museos"]=museos
    
    archivo=open("contaminacionMuseos.json","w",encoding="utf8")
    json.dump(museosJson, archivo, indent=2, ensure_ascii=False)
    archivo.close()

In [ ]:
estacionesLoc=leerEstacionesLocalizacion()
museos=leerMuseos()
contaminantes = leerEstacionesContaminacion()
cercanas=estacionesCercanas(estacionesLoc, museos)
mostrarMuseosContaminacion(cercanas, contaminantes)
escribirMuseos(cercanas,contaminantes)

#### Normas de entrega

* Fecha tope de entrega: 03/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.